In [ ]:
import torch
import pandas as pd
from scipy.io import loadmat
import numpy as np

In [ ]:
folder = "/Users/gr_1/Documents/Tai_py/"
s = loadmat(folder + 'sobjn')


In [ ]:
locs = loadmat(folder + 'locations')

In [ ]:
I,J = np.shape(s['sobj_conc'])

In [ ]:

sed = pd.DataFrame()

for i in range(I):
    for j in range(J):
        if not(np.size(s['sobj_conc'][i,j]) == 0):
            sedt = pd.DataFrame({'id':range(len(s['sobj_conc'][i,j]))})
            
            sedt['basin'] = i
            sedt['station'] = j
            try:
                sedt['datelin'] = s['sobj_dateslin'][i,j]
            except:
                sedt['datelin'] = np.nan
                print('here')
            sedt['month'] = s['sobj_dates'][i,j][:,0]
            sedt['day'] = s['sobj_dates'][i,j][:,1]
            
            sedt['conc'] = s['sobj_conc'][i,j]
            sedt['flow'] = s['sobj_flow'][i,j]
            try:
                f = np.where((locs['basin'] == i) & (locs['station'] == j))[0][0]

                sedt['x'] = locs['x'][f]
                sedt['y'] = locs['y'][f]
            except:
                sedt['x'] = np.nan
                sedt['y'] = np.nan
            sed = sed.append(sedt)
            
            
            
        

In [ ]:
sed = pd.DataFrame()
from IPython.core.debugger import set_trace
for i in range(I):
    for j in range(J):
        if not(np.size(s['sobj_conc'][i,j]) == 0):
            #sedt = pd.DataFrame({'id':range(len(s['sobj_conc'][i,j]))})
            ii = i+1
            ij = j+1
            sedt = pd.DataFrame({'id':[1]})
            sedt['basin'] = ii
            sedt['station'] = ij
            try:
                sedt['datelin'] = [s['sobj_dateslin'][i,j]]
            except:
                sedt['datelin'] = [np.nan]
                print('here')
            sedt['month'] = [s['sobj_dates'][i,j][:,0]]
            sedt['day'] = [s['sobj_dates'][i,j][:,1]]
            
            sedt['conc'] = [s['sobj_conc'][i,j]]
            sedt['flow'] = [s['sobj_flow'][i,j]]
            sedt['yeaer'] = (sedt['datelin'].values - 704919)/365.25 + 1930

            try:
                f = np.where((locs['basin'][0] == ii) & (locs['station'][0] == ij))[0]
                sedt['x'] = locs['x'][f]
                sedt['y'] = locs['y'][f]
            except:
                sedt['x'] = np.nan
                sedt['y'] = np.nan
            sed = sed.append(sedt)

In [ ]:
sed = pd.DataFrame()
from IPython.core.debugger import set_trace
for i in range(I):
    for j in range(J):
        if not(np.size(s['sobj_conc'][i,j]) == 0):
            #sedt = pd.DataFrame({'id':range(len(s['sobj_conc'][i,j]))})
            ii = i+1
            ij = j+1
            sedt = pd.DataFrame({'id':[1]})
            sedt['basin'] = ii
            sedt['station'] = ij
            try:
                sedt['datelin'] = [s['sobj_dateslin'][i,j]]
            except:
                
                sedt['datelin'] = [np.nan]
                print('here')
                continue
            try:
                sedt['month'] = [s['sobj_dates'][i,j][:,0]]
                sedt['day'] = [s['sobj_dates'][i,j][:,1]]
                yr = ((sedt['datelin'].values - 704919)/365.25 + 1930)
                sedt['year'] = yr
                sedt['conc_b'] = [s['sobj_conc'][i,j][sedt['year'].squeeze()<2009.67]]
                sedt['flow_b'] = [s['sobj_flow'][i,j][sedt['year'].squeeze()<2009.67]]
                sedt['year_b'] = [sedt['year'].squeeze()[sedt['year'].squeeze()<2009.67]]
                sedt['conc_b'][0][sedt['conc_b'][0]==0] = 1
                sedt['conc_a'] = [s['sobj_conc'][i,j][sedt['year'].squeeze()>=2009.67]]
                sedt['flow_a'] = [s['sobj_flow'][i,j][sedt['year'].squeeze()>=2009.67]]
                sedt['year_a'] = [sedt['year'].squeeze()[sedt['year'].squeeze()>=2009.67]]
                sedt['conc_a'][0][sedt['conc_a'][0]==0] = 1
                sedt['conc_a'][0][np.isnan(sedt['conc_a'][0])] = 1
                sedt['conc_b'][0][np.isnan(sedt['conc_b'][0])] = 1
                sedt['conc_a'][0][sedt['conc_a'][0]>=100000] = 1
                sedt['conc_b'][0][sedt['conc_b'][0]>=100000] = 1

            except:
                continue

            try:
                f = np.where((locs['basin'][0] == ii) & (locs['station'][0] == ij))[0]
                sedt['x'] = locs['x'][f]
                sedt['y'] = locs['y'][f]
            except:
                sedt['x'] = np.nan
                sedt['y'] = np.nan
            sed = sed.append(sedt)
sed.index = range(len(sed))

In [ ]:
sed.loc[0,'conc_b']

In [ ]:
def strcat(int1,int2):
    return str(int1)+'_'+str(int2)
sed['basta'] = list(map(strcat,sed['basin'],sed['station']))

In [93]:
import altair as alt
alt.data_transformers.disable_max_rows()
brush = alt.selection_single(nearest=True)

brush2 = alt.selection_interval(encodings=['x'])


points = alt.Chart().mark_circle().encode(
    x=alt.X('x',scale=alt.Scale(domain=[120, 122])),
    y=alt.Y('y',scale=alt.Scale(domain=[21, 26])),
    color=alt.condition(brush, 'id:O', alt.value('lightgray'), legend=None)
).add_selection(
    brush
).properties(
    width=200,
    height=400
)

bars = alt.Chart().mark_circle().transform_filter(brush).transform_flatten(['year_b','conc_b','flow_b']).encode(
    x=alt.X('flow_b:Q',scale=alt.Scale(zero=False,type='log',domain=(.010,100000))),
    y=alt.Y('conc_b:Q',scale=alt.Scale(zero=False,type='log',domain = (1,100000))),
    color=alt.value('#00FFFF')
    
).properties(
    width=500,
    height=300
    
)

bars2 = alt.Chart().mark_circle().transform_filter(brush).transform_flatten(['year_a','conc_a','flow_a']).encode(
    x=alt.X('flow_a:Q',scale=alt.Scale(zero=False,type='log',domain=(.010,100000))),
    y=alt.Y('conc_a:Q',scale=alt.Scale(zero=False,type='log',domain=(1,100000))),
    color=alt.value('#FF0000')
).properties(
    width=500,
    height=300
)


a=alt.vconcat(points,bars+bars2, data=sed)


In [96]:
a.save('/Users/gr_1/Desktop/tai_sed.html')

In [ ]:
import seaborn as sns
for a,b in sed.groupby('basta'):
    fig,ax = plt.subplots(figsize=(10,10))
    sns.boxplot(data=b,x='basta',y='conc')

In [ ]:
sed.loc[0,'conc_b']